<a href="https://colab.research.google.com/github/Junoflows/Deeplearning_From_Scatch2/blob/main/Chapter_5_%EC%88%9C%ED%99%98_%EC%8B%A0%EA%B2%BD%EB%A7%9D(RNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 5 순환 신경망(RNN)

+ 지금까지의 신경망은 흐름이 단방향인 신경망인 피드포워드라는 유형의 신경망이다.
+ 피드포워드 신경망은 구성이 단순하여 구조를 이해하기 쉽고 많은 문제에 응용할 수 있지만  
시계열 데이터를 잘 다루지 못한다는 단점이 있다.
+ 이에 대한 해결방안으로 순환 신경망(RNN)이 등장했다.

## 5.1 확률과 언어 모델

### 5.1.1 word2vec을 확률 관점에서 바라보다.

+ CBOW 모델을 복습해보자. $w_1, w_2, ..., w_t$ 라는 단어열로 표현되는 말뭉치를 생각하자.
+ t번째 단어를 타깃, 그 전후 단어를 맥락으로 생각하면 CBOW 모델은 $w_{t-1}, w_{t+1}$로부터 $w_t$를 추축하는 일을 수행한다.

<img src='http://drive.google.com/uc?export=view&id=1ddBdREgm03YCuVI1hMaEGGwOUfZp_ZTm' width = 550/><br>

+ $w_{t-1}, w_{t+1}$이 주어졌을 때 타깃이 $w_t$가 될 확률을 수식으로 나타내면 다음과 같다.

> $P(w_t|w_{t-1}, w_{t+1})$

+ CBOW 모델은 위 식에서 $w_{t-1}$과 $w_{t+1}$이 주어졌을 때 $w_t$가 일어날 확률인 사후 확률을 모델링한다.
+ 지금까지는 맥락을 항상 좌우 대칭으로 생각했는데 이번에는 맥락을 왼쪽 윈도우만으로 한정해보자.

<img src='http://drive.google.com/uc?export=view&id=1J6xuqAim7GjDpWkvJV4h6F323taDDdfo' width = 550/><br>

+ 위처럼 완쪽 두 단어만을 맥락으로 생각하면 CBOW 모델이 출력할 확률은 다음과 같다.

> $P(w_t|w_{t-2}, w_{t-1})$

+ 위 표기를 사용하면 CBOW 모델이 다루는 손실 함수를 아래와 같이 쓸 수 있는데 이는 교차 엔트로피 오차에 의해 유도된 결과이다.

> $L = -logP(w_t|w_{t-2}, w_{t-1})$

+ CBOW 모델의 학습으로 수행하는 일은 위 손실 함수를 최소화하는 가중치 매개변수를 찾는 것이다.
+ 이러한 가중치 매개변수가 발견되면 CBOW 모델은 맥락으로부터 타깃을 더 정확하게 추측할 수 있게 된다.
+ CBOW 모델을 학습시키는 본래 목적은 맥락으로부터 타깃을 정확하게 추측하는 것으로  
학습을 진행하면 단어의 의미가 인코딩된 단어의 분산 표현을 얻을 수 있다.
+ 이러한 맥락으로부터 타깃을 추측하는 것을 이용하는 것에 대해 언어 모델이 등장한다.

### 5.1.2 언어 모델

+ 언어 모델은 단어 나열에 확률을 부여한다.
+ 특정한 단어의 시퀀스에 대해 그 시퀀스가 일어날 가능성이 어느 정도인지 확률로 평가한다.

+ 언어 모델의 대표적인 예로 기계 번역과 음석 인식이 있다.
+ 음성 인식 시스템의 경우 사람의 음성으로부터 몇 개의 문장을 후보로 생성하고  
언어 모델을 사용하여 후보 문장이 문장으로써 자연스러운지를 기준으로 순서를 매길 수 있다.
+ 언어 모델은 새로운 문장을 생성하는 용도로도 이용할 수 있다.

+ 언어 모델의 수식에 대해 알아보자.
+ $w_1, ..., w_m$의 m개 단어로 된 문장을 생각해보면 단어가 $w_1, ..., w_m$ 순서로 출현할 확률을 $P(w_1, ..., w_m)$로 나타낸다.
+ 이 확률을 동시 확률이라 하는데 동시 확률은 사후 확률을 사용하여 아래와 같이 분해하여 쓸 수 있다.

$P(w_1, ..., w_m)$  
=  $P(w_m|w_1, ..., w_{m-1})P(w_{m-1}|w_1, ..., w_{m-2}) ... P(w_3|w_1, w_2)P(w_2|w_1)P(w_1)$  
= $\Pi_{t= 1}^m P(w_t|w_1, ..., w_{t-1})$

+ 동시 확률은 사후 확률의 총곱으로 나타낼 수 있다.
+ 위 수식은 확률의 곱셈정리로부터 유도된다.
+ 확률의 곱셈 정리는 다음과 같다.

$P(A, B) = P(A|B)P(B)$

+ 이를 이용해 m개 단어의 동시 확률 $P(w_1, ..., w_m)$ 을 사후 확률로 나타낼 수 있다.
+ $w_1, ..., w_{m-1}$을 A로 나타낸다.  

$P(w_1, ..., w_m) = P(A, w_m) = P(w_m|A)P(A)$  

+ 한번 더 $A(w_1, ..., w_m)$ 에 대해 같은 식 변형을 하자.

$P(A) = P(w_1, ..., w_{m-2}, w_{m-1}) = P(A', w_{m-1}) = P(w_{m-1}|A')P(A')$  

+ 이처럼 단어 시퀀스를 하나씩 줄여가면서 사후 확률로 분해하면 아래와 같은 식을 유도할 수 있다.

$P(w_1, ..., w_m)$  
=  $P(w_m|w_1, ..., w_{m-1})P(w_{m-1}|w_1, ..., w_{m-2}) ... P(w_3|w_1, w_2)P(w_2|w_1)P(w_1)$  
= $\Pi_{t= 1}^m P(w_t|w_1, ..., w_{t-1})$

+ 주목할 것은 사후 확률은 타깃 단어보다 왼쪽에 있는 모든 단어를 맥락으로 했을 때의 확률이다.

<img src='http://drive.google.com/uc?export=view&id=148lYmGrON1yY-kjmhjHn-Edu7fXvHM96' width = 550/><br>

+ 우리의 목표는 $P(w_t|w_1, ..., w_{t-1})$의 확률을 얻는 것으로 이것을 계산할 수 있다면 언어 모델의 동시 확률인 $P(w_1, ..., w_m)$을 구할 수 있다.

### 5.1.3 CBOW 모델을 언어 모델로?

+ word2vec의 CBOW 모델을 언어 모델에 적용하려면 어떻게 해야할까?
+ 맥락의 크기를 특정 값으로 한정하여 근사적으로 나타낼 수 있다.

$P(w_1, ..., w_m) = \Pi_{t= 1}^m P(w_t|w_1, ..., w_{t-1}) ≃ \Pi_{t= 1}^m P(w_t|w_{t-2}, w_{t-1})$

+ 맥락을 왼쪽 2개의 단어로 한정하여 근사적으로 나타낼 수 있다.
+ 맥락의 크기는 임의의 길이로 설정할 수 있으나 결국 특정 길이로 고정된다.
+ 이는 임의로 설정한 맥락의 크기보다 왼쪽에 있는 단어의 정보는 무시되는 문제가 있다.

<img src='http://drive.google.com/uc?export=view&id=148lYmGrON1yY-kjmhjHn-Edu7fXvHM96' width = 550/><br>

<img src='http://drive.google.com/uc?export=view&id=16qgq3fkiC7gdf2DQGCs9Ob8FFz2T3J6v' width = 550/><br>

+ 위 문제의 정답은 Tom이지만 CBOW 모델의 맥락이 10개까지라면 제대로 답할 수 없다.
+ 맥락의 크기를 키우면 해결할 수 있지만 CBOW 모델에서는 맥락 안의 단어 순서가 무시된다는 한계가 있다.

<img src='http://drive.google.com/uc?export=view&id=1b3fRkHYT5BQlCFMZVtI4CZXEBDnQOx6K' width = 550/><br>

+ 왼쪽 그림과 같이 CBOW 모델의 은닉층에서는 단어 벡터들이 더해지므로 단어 순서는 무시된다.
+ 단어 순서도 고려한 모델이 이상적이므로 오른쪽 그림처럼 맥락의 단어 벡터를 은닉층에서 연결하는 방식을 생각할 수 있다.
+ 하지만 이 방식은 맥락의 크기에 비례해 가중치 매개변수도 늘어나므로 좋은 방법은 아니다.

+ 이 문제의 해결 방법으로 순환 신경망(RNN)을 사용한다.
+ RNN은 맥락의 길이의 상관없이 맥락의 정보를 기억하므로 아무리 긴 시계열 데이터에도 사용할 수 있다.

## 5.2 RNN이란

### 5.2.1 순환하는 신경망

+ 순환한다는 의미는 반복해서 되돌아감을 의미하여 순환하기 위해서는 닫힌 경로가 필요하다.+
+ 닫힌 경로가 존재하면 데이터가 순환하면서 정보가 끊임없이 갱신된다.
+ RNN에 이용되는 계층을 RNN 계층이라 부르며 다음과 같이 그린다.

<img src='http://drive.google.com/uc?export=view&id=1ISzXbsCFjKov6CBxjt1NZpDZef_GKH0_' width = 550/><br>

+ RNN 계층은 순환하는 경로를 따라 데이터를 계층 안에서 순환시킬 수 있다.
+ $X_t$에서의 t는 시각을 뜻하며 시계열 데이터 $(X_0, X_1, ..X_t, ...)$가 RNN 계층에 입력됨을 나타내며 $(h_0, h_1, ..h_t, ...)$가 출력된다.

+ $X_t$는 벡터라고 가정하며 문장을 다루는 경우는 각 단어의 분산 표현이 $X_t$가 되며 이 분산 표현이 순서대로 하나씩 RNN 계층에 입력된다.

+ 위 그림의 순환 구조를 자세히 살펴보기전에 RNN 계층을 그리는 방식을 변경하자.

<img src='http://drive.google.com/uc?export=view&id=1vSdlf555rgIhozppP87ZgZ_p5sCRiYuH' width = 550/><br>

+ 앞으로 계층을 그릴 때 아래에서 위로 흐르도록 그린다. (순환 구조를 펼칠 때 계층을 양옆으로 펼치기 위함)

### 5.2.2 순환 구조 펼치기

+ RNN 계층의 순환 구조에 대해 자세하게 살펴보자.
+ RNN의 순환 구조를 펼쳐보자.

<img src='http://drive.google.com/uc?export=view&id=1svgw11gijDhYIcOcH5Zb8ECjeJA9hZJp' width = 550/><br>

+ RNN 계층의 순환 구조를 펼치면서 오른쪽으로 성장하는 긴 신경망으로 만들 수 있는데 이는 피드포워드 신경망과 같은 구조다.
+ 다만 다수의 RNN 계층 모두가 실제로는 같은 계증인 것이 지금까지의 신경망과는 다르다.

+ RNN 계층은 그 계층으로의 입력과 1개 전의 RNN 계층으로부터 출력을 받는다.
+ 이를 수행하는 계산 수식은 다음과 같다.

__$h_t$__ = tanh$(h_{t-1}W_h + X_tW_x + b)$

+ RNN에서의 가중치는 입력 $x$를 출력 $h$로 변환하기 위한 $W_x$와 1개의 RNN출력을 다음 시각의 출력으로 변환하기 위한 $W_h$가 있다.
+ 편향 $b$와 행벡터 $h_{t-1}, X_t$가 있다.

+ 행렬 곱을 계산하고 그 합을 tanh 함수를 이용해 변환하여 $h_t$를 출력한다.
+ $h_t$는 다른 계층을 향해 위쪽으로 출력되면서 다음 시각의 RNN 계층을 향해 오른쪽으로도 출력된다.
+ $h_t$는 $h_{t-1}$ 에 기초해 계산됨을 알 수 있고 이는 RNN은 $h$라는 상태를 가지고 있으며 위 식의 형태로 갱신된다고 해석할 수 있다.
+ 많은 경우 펼쳐진 RNN 계층을 아래의 왼쪽처럼 그린다.

<img src='http://drive.google.com/uc?export=view&id=1RLK7eN96r-AHJtjbfwgpG_O-e_7f69mY' width= 550/><br>

+ 왼쪽그림은 RNN 계층에서 나가는 두 화살표가 똑같은 데이터가 복사돼 분기된다는 사실이 잘 드러나지 않으므로 오른쪽 그림의 방식으로 사용한다.

### 5.2.3 BPTT

+ RNN 계층은 가로로 펼친 신경망으로 생각할 수 있으므로 학습도 보통의 신경망과 같은 순서로 진행할 수 있다.

<img src='http://drive.google.com/uc?export=view&id=1hZ0nDbP5kfYnxkwV5DiPnvg4aI2EmGx4' width = 550/><br>

+ 순환 구조를 펼친 후의 RNN에는 오차역전파법을 적용할 수 있다.
+ 먼저 순전파를 수행하고 역전파를 수행하여 원하는 기울기를 구할 수 있다.
+ 시간 방향으로 펼친 신경망의 오차역전파법이란 뜻의 BPTT(Backpropagation Through Time)이라고 한다.

+ BPTT를 이용하여 RNN을 학습하기엔 긴 시계열 데이터를 학습할 때의 문제가 있다.
+ 시계열 데이터의 시간 크기가 커지는 것에 비례하여 BPTT가 소비하는 컴퓨팅의 자원도 증가하기 때문에 역전파 시의 기울기가 불안정해진다.

### 5.2.4 Truncated BPTT

+ 큰 시계열 데이터를 취급할 때는 시간축 방향으로 너무 길어진 신경망을 적당한 지점에서 잘라  
작은 신경망 여러 개로 만들고 각각에서 오차역전파법을 수행한다.
+ 이를 Truncated BPTT 라고 한다.

+ Truncated BPTT에서는 순전파의 연결은 유지하면서 역전파의 연결만 끊어야 한다.
+ Truncated BPTT의 구체적인 예를 살펴보자.

+ 길이가 1000인 시계열 데이터가 존재한다고 하자. 자연어 문제라면 단어 1000개의 말뭉치에 해당한다.
+ 이 시계열 데이터로 RNN 계층을 펼치면 가로로 1000개나 늘어선 신경망이 된다.
+ 이는 계산량과 메모리 사용량 등의 문제가 되며 계층이 길어짐에 따라  
신경망을 하나 통과할 때마다 기울기가 작아져서 t시각의 역전파가 되기 전에 0으로 소멸할 수도 있다.
+ 따라서 가로로 길게 뻗은 신경망의 역전파에서는 적당한 길이로 끊는다.

<img src='http://drive.google.com/uc?export=view&id=10dd0FxLn7SPF41ne8n28C9Ocm4p8cjzD' width = 550><br>

+ 위 그림에서는 RNN 계층을 길이 10개 단위로 역전파의 연결을 끊었다.
+ 이렇게 하면 각각의 블록 단위로 미래의 불록과는 독립적으로 오차역전파법을 할 수 있다.
+ 여기서 중요한 점은 역전파는 끊어지지만 순전파는 끊어지지 않는다는 것이다.

+ Truncated BPTT 방식으로 RNN을 학습시켜보자.
+ 우선 첫 번째 블록 입력 데이터를 RNN 계층의 제공한다.

<img src='http://drive.google.com/uc?export=view&id=1RAhZBNmC81QtRiRnZ24wkakWkoupLxE_' width = 550/><br>

+ 위 그림에서 보듯 먼저 순전파를 수행하고 역전파를 수행하여 원하는 기울기를 구한다.
+ 이어서 다음 블룩의 입력 데이터를 입력해 오차역전파법을 수행한다.

<img src='http://drive.google.com/uc?export=view&id=1hq4APf0rF23I9VN-wrEcnUls7k1f9pK5' width = 550/><br>

+ 여기서도 순전파를 수행하고 역전파를 수행한다.
+ 이번 순전파 계산에는 앞 블록의 마지막 은닉 상태인 $h_9$가 필요하다는 것으로 이로인해 순전파는 계속 연결될 수 있다.
+ 같은 방식으로 3번째 블록을 대상으로 학습을 수행한다.

<img src='http://drive.google.com/uc?export=view&id=170GFe9OqVUtsL8JSAlT3CG58vAREmgBr' width = 550/><br>

+ 위처럼 Truncated BPTT에서는 데이터를 순서대로 입력해 학습하며 순전파의 연결을 유지하면서 블록 단위로 오차역전파법을 적용할 수 있다.

### 5.2.5 Truncated BPTT의 미니배치 학습

+ 학습을 위해 미니배치를 수행하기 때문에 데이터를 주는 시작 위치를 각 미니배치의 시작 위치로 옮겨줘야 한다.
+ 옮긴다를 이해하기 위해 길이가 1000인 시계열 데이터를 시각의 길이를 10개 단위로 잘라 학습하는 경우를 생각해보자.
+ 미니배치를 2개로 구성해 학습하기 위해서 첫 번쨰 미니배치 때는 처음부터 순서대로 데이터를 제공하고,  
두 번째 미니배치 때는 500번째의 데이터를 시작 위치로 정하고 그 위치부터 다시 순서대로 데이터를 제공한다.

<img src='http://drive.google.com/uc?export=view&id=1wrNeORcoITAbLjpMabKtrE_xNi9zgYB3' width = 550/><br>

+ 첫 번째 미니배치 원소는 $x_0, ..., x_9$가 되고 두 번째 미니배치 원소는 $x_{500}, ..., x_{509}$가 된다.
+ 이 미니배치 데이터를 RNN의 입력 데이터로 사용해 학습을 수행한다.
+ 다음에 넘길 데이터는 각각 시계열 데이터의 10~19번째 데이터와 510 ~ 519번째 데이터가 된다.

+ 이처럼 미니배치 학습을 수행할 때는 각 미니배치의 시작 위치를 오프셋으로 옮겨준 후 순서대로 제공한다.
+ 또한 데이터를 순서대로 입력하다가 끝에 도달하면 다시 처음부터 입력하도록 한다.
+ Truncated BPTT에서 '데이터를 순서대로 제공하기'와 '미니배치별로 데이터를 제공하는 시작위치를 옮기기'를 주의해야한다.

## 5.3 RNN 구현

+ 가로 방향으로 성장한 신경망을 구현하기 위해 Truncated BPTT 방식으로 신경망을 만든다.

<img src='http://drive.google.com/uc?export=view&id=1HKHo5WtKf5WG-5Sy-GAK3UhCwpOVaSde' width = 550/><br>

+ 우리가 다룰 신경망의 길이가 T인 시계열 데이터를 받고 각 시각의 은닉 상태를 T개 출력한다.
+ 모듈화를 생각해 옆으로 성장한 신경망을 하나의 계층으로 구현해보자.

<img src='http://drive.google.com/uc?export=view&id=1_4YZ433LxOKSsEuDznWioGWnFxsDA8yY' width = 550/><br>

+ 위와 같이 상하 방향의 입력과 출력을 각각 하나로 묶으면 옆으로 늘어선 일련의 계층을 하나의 계층으로 간주할 수 있다.
+ 즉 $xs=(x_0,...,x_{T-1})$을 입력하면 $hs = (h_0, ..., h_{T-1})$를 출력하는 단일 계층으로 볼 수 있다.
+ Time RNN 계층에서 한 단계의 작업을 수행하는 계층을 RNN 계층이라 하고  
T개 단계분의 작업을 한꺼번에 처리하는 계층을 Time RNN 계층이라 한다.

### 5.3.1 RNN 계층 구현